# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import csv
import gmaps
import os
import json
# Import API key
from api_keys import g_key

In [2]:
vacation_df = pd.read_csv("vacation.csv")
vacation_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,fortuna,1.0,US,2019-09-26 15:36:26,100.0,40.60,-124.16,55.00,20.09
1,1,leningradskiy,0.0,RU,2019-09-26 15:33:59,72.0,69.38,178.42,25.33,7.02
2,2,upernavik,100.0,GL,2019-09-26 15:33:54,58.0,72.79,-56.15,37.03,8.48
3,3,mataura,74.0,NZ,2019-09-26 15:33:33,81.0,-46.19,168.86,48.00,15.01
4,4,ushuaia,75.0,AR,2019-09-26 15:31:39,92.0,-54.81,-68.31,50.00,13.87


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Store latitude and longitude in locations
locations = vacation_df[["Latitude", "Longitude"]]
vacation_df.replace("", np.nan, inplace = True)
vacation_df = vacation_df.dropna()

# Fill NaN values and convert to float
humidity = vacation_df["Humidity"].astype(float)

In [7]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
vacation_df["Hotel Name"] = ""
vacation_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,0,fortuna,1.0,US,2019-09-26 15:36:26,100.0,40.60,-124.16,55.00,20.09,
1,1,leningradskiy,0.0,RU,2019-09-26 15:33:59,72.0,69.38,178.42,25.33,7.02,
2,2,upernavik,100.0,GL,2019-09-26 15:33:54,58.0,72.79,-56.15,37.03,8.48,
3,3,mataura,74.0,NZ,2019-09-26 15:33:33,81.0,-46.19,168.86,48.00,15.01,
4,4,ushuaia,75.0,AR,2019-09-26 15:31:39,92.0,-54.81,-68.31,50.00,13.87,


In [9]:
vac_file=vacation_df.loc[(vacation_df['Max Temp']>=70)&(vacation_df['Max Temp']<=80)]
wind_file=vac_file.loc[(vac_file['Wind Speed']<=10)]
file=wind_file.loc[(wind_file['Cloudiness']==0)]
file.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
5,5,hermanus,0.0,ZA,2019-09-26 15:33:27,55.0,-34.42,19.24,70.69,4.25,
70,70,sabzevar,0.0,IR,2019-09-26 15:36:32,13.0,36.21,57.68,73.39,7.07,
78,78,ravar,0.0,IR,2019-09-26 15:36:33,15.0,31.27,56.81,75.91,7.67,
80,80,kruisfontein,0.0,ZA,2019-09-26 15:31:49,28.0,-34.00,24.73,79.87,4.21,
128,128,carnarvon,0.0,ZA,2019-09-26 15:33:48,8.0,-30.97,22.13,76.45,8.50,


In [10]:
hotel_df = file.reset_index(drop=True)
hotel_df
hotel_df["Hotel Name"] = ""
hotel_df
#hotel_df = hotel_df.loc[:, ~hotel.columns.str.contains()]
hotel_df

params = {
  "radius": 5000,
  "types": "hotel",
  "keyword": "hotels",
  "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAK9e0FT-gyOTqoJRkQ89LKFnihpxEGOba7H3hiA81lDN_VWpDII6KuaC1JR_nd688ls6Ly_xdWdEVkiqstIyvmseJ2OnVBuU3UrTqI-RJAhqHDuQAhW4ldzURsOUmHsgtaxkC2yLw8ZLSwg7MhfCo5O6OiyffhdScwjBwOqoXL0x3A-iBYrk_-ti4HHDBO2Z2wVbMI6fzr_a_4CsPUwjKzhWjuZ92G0Fer-ZK7WDGLUQrbTn7kEPVOV20m1NKnNZ2qugTYYmTWkjj0lrDvmb6YGhk59N-EmD5bZEZvkV6nSNMqVE8wtsASz0cOqwiQyBDcKJmXZ8_xXV-I1VxTGz6c4lysPalH9kbrDS_oujZVC-ix8fr3ESOpdRD0MBlHNBCVJwKVpinhXy4YKsjWJ8KSUSEPmQVE4Xssxp3pbYZhnMLQoaFJ-x_0ue8SVx5S9wuHU6VOinXK7j",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.4212214,
                    "lng": 19.2401906
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.41989117010728,
                        "lng": 19.24153182989272
                    },
                    "southwest": {
                        "lat": -34.42259082989272,
                    

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 36.2122736,
                    "lng": 57.66857839999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.21357502989273,
                        "lng": 57.66996477989272
                    },
                    "southwest": {
                        "lat": 36.21087537010729,
                        "lng": 57.66726512010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f99d00db87b15e1d810227257b41760f63a10c76",
            "name": "Sabzevar Tourism hotel",
            "opening_hours": {
                "open_now": false
            },
            "photos": [
                {
                    "height": 600,
                    "html_attributions": [
           

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.031,
                    "lng": 24.7679
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.02964552010727,
                        "lng": 24.76925142989272
                    },
                    "southwest": {
                        "lat": -34.03234517989272,
                        "lng": 24.76655177010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "89af086f477dd8d6f1b8914161439b3ca5dfb932",
            "name": "Humansdorp Boutique Hotel",
            "opening_hours": {
                "open_now": false
            },
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
                  

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 31.6225645,
                    "lng": 55.4203901
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.62353817989271,
                        "lng": 55.42176172989272
                    },
                    "southwest": {
                        "lat": 31.62083852010727,
                        "lng": 55.41906207010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "82c5039d6fd7fd6b2efcdf4e8a5742a791af8f83",
            "name": "Nikan Hotel",
            "photos": [
                {
                    "height": 2988,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/100733863105408301357/photos\">Mehdi

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAADbcA8VgPPaBB0eUalJzHdUoj2GFxg0wB5MWtkxYBWVfgeLW3n5ViVzVVjUlFu_SZRS3MOJxS8U73mtDXRhDBI0VR1lUqwj0Y5mvc85XX5Ik4P2TPNjntfLCW7rYYmUAMO6QXXfDembtPdnLtfUBrlZyPHJfJ-nD2OZn9mVzx-gbTlQbdFBUrtyrDP8cfdYMGQY-znpst0MXfLqUEkiIgrEa0Lq_u_a86kKFF6xIWi_g3po53_GTBZgsmQwScuLOk-S9IGbUEKxpl7k2uL8IHBaL5nOat0Dg8Cuzo6qBusujKyqq9USWtSUuPWSCQ4X2GyzsDIz86p2YtFSPmVmbTHFvWSz1V2I0NJjS9Yx9cUjMKbRs6FjVB-dXcqkHDSq5aY__CynMN_wIe-TNfESqeGwSEDKbFGubmapg6w1sxnrSassaFOtyXM9_jAj1-VC9efTfBdYKF4bC",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 29.697392,
                    "lng": 120.23984
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.69871887989272,
                        "lng": 120.2411404798927
                    },
                    "southwest": {
                        "lat": 29.69601922010727,
                        "

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -17.5395619,
                    "lng": -149.3664944
                },
                "viewport": {
                    "northeast": {
                        "lat": -17.53848722010728,
                        "lng": -149.3654216701073
                    },
                    "southwest": {
                        "lat": -17.54118687989272,
                        "lng": -149.3681213298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "fa9280a3550375e269adf9f7411f5dd4611e33a5",
            "name": "Le Rocher de Tahiti",
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/109053425642595887131

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Save Data to csv
hotel_df.to_csv("Hotels_Output.csv")
# Visualize to confirm data appears
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,5,hermanus,0.0,ZA,2019-09-26 15:33:27,55.0,-34.42,19.24,70.69,4.25,Windsor Hotel
1,70,sabzevar,0.0,IR,2019-09-26 15:36:32,13.0,36.21,57.68,73.39,7.07,Sabzevar Tourism hotel
2,78,ravar,0.0,IR,2019-09-26 15:36:33,15.0,31.27,56.81,75.91,7.67,Hotel sperlous
3,80,kruisfontein,0.0,ZA,2019-09-26 15:31:49,28.0,-34.00,24.73,79.87,4.21,Humansdorp Boutique Hotel
4,128,carnarvon,0.0,ZA,2019-09-26 15:33:48,8.0,-30.97,22.13,76.45,8.50,Lord Carnarvon Guesthouse
5,149,saldanha,0.0,PT,2019-09-26 15:33:41,27.0,41.42,-6.55,75.91,2.10,
6,153,lagoa,0.0,PT,2019-09-26 15:33:16,60.0,37.14,-8.45,79.00,8.05,Colina Hotels & Resorts
7,155,bafq,0.0,IR,2019-09-26 15:36:40,15.0,31.60,55.40,76.27,4.32,Nikan Hotel
8,177,esfarayen,0.0,IR,2019-09-26 15:36:42,19.0,37.08,57.51,78.80,9.17,مهمانپذیر آفتاب
9,299,zhuji,0.0,CN,2019-09-26 15:36:55,77.0,29.72,120.23,73.99,4.47,Howard Johnson Tongfang Plaza Zhuji


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)

# Add the layer to the map

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))